In [1]:
import pandas as pd
import yfinance as yf

from pyecharts.charts import Line, Bar, Grid
from pyecharts import options as opts

In [2]:
def nearest_half(df):
    return df.mul(2).round().div(2)

In [3]:
ticks=['SPY']
data=yf.download(ticks,interval='15m',start="2020-05-11", auto_adjust=True, prepost=False)
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2020-06-26 14:45:00-04:00,300.190002,300.799988,299.850006,300.470001,5299713
2020-06-26 15:00:00-04:00,300.470001,301.035004,299.989990,300.649994,4389264
2020-06-26 15:15:00-04:00,300.640015,300.839996,299.660004,300.345001,5946555
2020-06-26 15:30:00-04:00,300.339996,301.000000,299.420013,300.989990,5722038
2020-06-26 15:45:00-04:00,300.980011,301.459991,299.739990,299.950012,16630779


In [4]:
df = data.copy()
df[['Open', 'High', 'Low', 'Close']] = nearest_half(df[['Open', 'High', 'Low', 'Close']])
df

,Open,High,Low,Close,Volume
Datetime,,,,,
2020-05-11 09:30:00-04:00,290.5,291.0,290.0,291.0,7157743
2020-05-11 09:45:00-04:00,291.0,291.0,290.5,290.5,4106553
2020-05-11 10:00:00-04:00,290.5,291.5,290.5,291.0,3386071
2020-05-11 10:15:00-04:00,291.0,291.0,290.5,291.0,2404748
2020-05-11 10:30:00-04:00,291.0,291.5,290.5,291.0,2829116
...,...,...,...,...,...
2020-06-26 14:45:00-04:00,300.0,301.0,300.0,300.5,5299713
2020-06-26 15:00:00-04:00,300.5,301.0,300.0,300.5,4389264
2020-06-26 15:15:00-04:00,300.5,301.0,299.5,300.5,5946555


In [5]:
perfil = pd.concat([df[[col,'Volume']].set_index(col) for col in ['Open', 'High', 'Low', 'Close']])
perfil = perfil.groupby(perfil.index).sum().div(4)
perfil

,Volume
273.0,3122817.50
276.5,3371988.50
277.0,4433686.50
277.5,6329091.75
278.0,1594349.25
...,...
321.0,34716124.50
321.5,33954014.25
322.0,30360822.75
322.5,9689280.00


In [6]:
bars = (Bar(init_opts=opts.InitOpts(theme='vintage'))
        .add_xaxis(perfil.index.tolist())
        .add_yaxis('Profile', perfil['Volume'].tolist())
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(xaxis_opts=opts.AxisOpts(
            is_show=False))
        .reversal_axis()
          )

In [7]:
line = (Line(init_opts=opts.InitOpts(theme='vintage'))
        .add_xaxis(data.index.strftime("%d%m %H:%M").tolist())
        .add_yaxis('Close', data['Close'].round(2).tolist(),
                  label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
            yaxis_opts=opts.AxisOpts(
                is_show=False,
                min_=perfil.index.min(),
                max_=perfil.index.max()),
            legend_opts=opts.LegendOpts(pos_left='25%'))
       )

In [8]:
grid = (
    Grid(init_opts=opts.InitOpts(width="800px", height="400px", theme="vintage"))
    .add(bars, grid_opts=opts.GridOpts(pos_right="40%"))
    .add(line, grid_opts=opts.GridOpts(pos_left="10.2%"))
)
grid.render_notebook()

### Ahora todo junto como una función personalizable

In [9]:
def nearest_half(df):
    return df.mul(2).round().div(2)

def profile(data, line_col='Close', volume_col='Volume', ohlc_cols=None,
            time_format="%d%m %H:%M",
            width="800px", height="400px", theme='vintage'):
    df = data.copy()
    if ohlc_cols is None:
        ohlc_cols = ['Open', 'High', 'Low', 'Close']
    df[ohlc_cols] = nearest_half(df[ohlc_cols])

    perfil = pd.concat([df[[col, volume_col]].set_index(col) for col in ohlc_cols])
    perfil = perfil.groupby(perfil.index).sum().div(4)

    bars = (Bar()
            .add_xaxis(perfil.index.tolist())
            .add_yaxis('Profile', perfil[volume_col].tolist())
            .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(xaxis_opts=opts.AxisOpts(
                is_show=False))
            .reversal_axis()
              )

    line = (Line()
            .add_xaxis(data.index.strftime(time_format).tolist())
            .add_yaxis(line_col, data[line_col].round(2).tolist(),
                      label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(
                xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
                yaxis_opts=opts.AxisOpts(
                    is_show=False,
                    min_=perfil.index.min(),
                    max_=perfil.index.max()),
                legend_opts=opts.LegendOpts(pos_left='35%'))
           )

    grid = (
        Grid(init_opts=opts.InitOpts(width=width, height=height, theme=theme))
        .add(bars, grid_opts=opts.GridOpts(pos_right="40%"))
        .add(line, grid_opts=opts.GridOpts(pos_left="10.2%"))
    )
    
    return  grid.render_notebook()

In [10]:
profile(data)

In [398]:
profile(data, 'Open', width= 1000, theme='macarons')